In [66]:
import pandas as pd 
import numpy as np

#Loading data 
histo=pd.read_csv("historical_data.csv")
senti=pd.read_csv("fear_greed_index.csv")

histo.head(5)
senti.head(5)

#for number of column and rows 
print("Histo shape:", histo.shape)
print("Sentiment shape:", senti.shape)

print(histo.info())
print(senti.info())

#for missing value which is 0 in both case
print(histo.isnull().sum())
print(senti.isnull().sum())

#for duplicate rows which is zero
print("Histo duplicates:", histo.duplicated().sum())
print("Sentiment duplicates:", senti.duplicated().sum())

#if want duplicate values in each column then
for col in histo.columns:
    print(col, ":", histo[col].duplicated().sum())
for col in senti.columns:
    print(col, ":", senti[col].duplicated().sum())

#now cleaning column names
histo.columns = histo.columns.str.strip()
senti.columns = senti.columns.str.strip()

#converting Timestamp
histo['Timestamp IST'] = pd.to_datetime(histo['Timestamp IST'], dayfirst=True, errors='coerce')
histo['date'] = histo['Timestamp IST'].dt.date

senti['date'] = pd.to_datetime(senti['date'])
senti['date'] = senti['date'].dt.date


#merging data for furthur insights
merged = pd.merge(histo, senti, on='date', how='inner')

#for viewing merged dataset
merged.head(5)

#cleaning merge data coulmn's name
merged.columns = merged.columns.str.strip()

# win column
merged['win'] = merged['Closed PnL'] > 0

# daily pnl per account
daily_pnl = merged.groupby(['date', 'Account'])['Closed PnL'].sum().reset_index()

# win rate
win_rate = merged.groupby('Account')['win'].mean()

# avg trade size
avg_size = merged.groupby('Account')['Size USD'].mean()

# trades per day
trades_per_day = merged.groupby('date').size()

# long/short ratio
long_short = pd.crosstab(merged['classification'], merged['Side'], normalize='index')

# leverage distribution
if 'Leverage' in merged.columns:
    leverage_dist = merged['Leverage'].describe()



###################################################----ANALYSIS PART-----------###################################################
#Q1: Performance vs Fear/Greed
# pnl
pnl_sent = merged.groupby('classification')['Closed PnL'].mean()

# win rate
win_sent = merged.groupby('classification')['win'].mean()

# drawdown proxy (negative pnl)
drawdown = merged[merged['Closed PnL'] < 0].groupby('classification')['Closed PnL'].mean()

#FOR VISUALIZING Performance vs Fear/Greed

import matplotlib.pyplot as plt

pnl_sent.plot(kind='bar', title='Avg PnL by Sentiment')
plt.show()

win_sent.plot(kind='bar', title='Win Rate by Sentiment')
plt.show()


#Q2: Behavior Change

# trade frequency
freq = merged.groupby(['date', 'classification']).size().reset_index(name='trades')
freq.groupby('classification')['trades'].mean()

# position size
size_sent = merged.groupby('classification')['Size USD'].mean()

# long/short
ls = pd.crosstab(merged['classification'], merged['Side'], normalize='index')

# leverage
if 'Leverage' in merged.columns:
    lev = merged.groupby('classification')['Leverage'].mean()


#Q3 Segmentation 

   #3(a) High vs Low Leverage

if 'Leverage' in merged.columns:
    merged['lev_group'] = pd.qcut(merged['Leverage'], 2, labels=['Low', 'High'])
    lev_analysis = merged.groupby('lev_group')['Closed PnL'].mean()


    #3(b) Frequent vs Infrequent
    
trade_count = merged.groupby('Account').size()
median = trade_count.median()

merged['freq_type'] = merged['Account'].map(
    lambda x: 'Frequent' if trade_count[x] > median else 'Infrequent'
)

freq_analysis = merged.groupby('freq_type')['Closed PnL'].mean()



    #3(c)Consistent vs Inconsistent

win_rate_acc = merged.groupby('Account')['win'].mean()

merged['consistency'] = merged['Account'].map(
    lambda x: 'Consistent' if win_rate_acc[x] > 0.6 else 'Inconsistent'
)

cons_analysis = merged.groupby('consistency')['Closed PnL'].mean()

print(cons_analysis)



##################################-----------------------INSIGHTS----------------------------------#############################

#1 Performance Analysis

From the analysis, it is observed that traders tend to perform better during Greed periods.
The average PnL and win rate are higher compared to Fear periods.
This suggests that market conditions with positive sentiment provide better opportunities for profitable trades.


#2 Behavioral Changes

Trader behavior changes depending on market sentiment. During Greed periods, traders execute more trades and take larger position sizes,
indicating a more aggressive trading approach. In contrast, during Fear periods, trading activity and position sizes are relatively lower, 
reflecting cautious behavior.

#3 Segmentation Analysis

Different groups of traders show varying performance patterns. High-leverage traders tend to have more volatile PnL, indicating higher risk.
On the other hand, traders with consistent win rates achieve more stable returns compared to inconsistent traders,
suggesting that consistency is an important factor in long-term performance.


############################----------------Actionable Strategies--------------------######################################
1. Strategy for Fear Periods

During Fear periods, traders should adopt a more conservative approach. Reducing leverage and limiting position sizes can help manage risk,
as overall performance and win rates tend to be lower in such market conditions.

2. Strategy for Greed Periods

During Greed periods, traders can increase their trading activity and take advantage of positive market trends.
Taking more long positions and increasing trade frequency can improve profitability, but proper risk management is still necessary to avoid large losses.

3. Trader Selection Strategy

It is advisable to focus on traders who have a consistent win rate rather than those who rely on high leverage or high-frequency trading.
Consistent traders are more likely to generate stable returns over time and carry lower risk.


#########################-----------------------SUMMARY---------------#######################################

    Conducted sentiment-driven trading analysis by integrating market sentiment with historical trading data, 
    identifying performance differences, behavioral changes, and actionable trading strategies.











SyntaxError: invalid syntax (3811703739.py, line 156)